In [66]:
import string

def remove_chars_from_text(text, chars):
    # return "".join([ch for ch in text if ch not in chars])
    content = ''
    for ch in text:
        if ch not in chars:
            content = content + ''.join(ch)
        else:
            content = content + ''.join(' ')
    return content

import re

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, ' ', data)
spec_chars = string.punctuation + '\r' + '\n\xa0«»\t—…' 


In [67]:
def predict_contry (Text_post):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.model_selection import cross_val_score
    import pickle

    
    with open('contry_post-model_ngram_rf_RandomForest_V1_001.bin', 'rb') as f_out:
        rf_RandomForest, vocabl=pickle.load( f_out)
    x_temp = []
    content = str(Text_post).lower()  
    content = remove_emojis(content)
    content = remove_chars_from_text(content, spec_chars)
    content = remove_chars_from_text(content, string.digits)
    for i in range(1, 10):
        content = content.replace('  ', ' ')
    x_temp.append(content)


    count_ngram = CountVectorizer(ngram_range=(2,2))
    X_train_transformed_ngram = count_ngram.fit_transform(x_temp)
    vocab = count_ngram.vocabulary_
    #print(len(vocab))
    #print(vocabl[:20])
    corpus_with_ngram= []
    for text in x_temp:    
        for item in vocab:
            if item.replace(' ', '_') in vocabl:
                text = text.replace(item, item.replace(' ', '_'))
                #print(item)
        corpus_with_ngram.append(text)
    
    #print (corpus_with_ngram)
    count = CountVectorizer(vocabulary = vocabl)
    X_t = count.transform(corpus_with_ngram)
    accuracy = rf_RandomForest.predict_proba(X_t)
    ix = accuracy.argmax(1).item()
    # print(f'predicted class = {log_clf.classes_[ix]} and confidence = {accuracy[0,ix]:.2%}')
    predicted_class =rf_RandomForest.classes_[ix]
    confidence = accuracy[0,ix]    
    return predicted_class, confidence

In [68]:


def get_telegram_adv(user_name, Dir_Path= 'Set_A', iter_number=65, text_limit=True):
    '''
    :param user_name - Название телеграм канала:
    :param iter_number - Количество иттераций по дате публикации постов каждый шаг 20 постов max:
    :return:
    :text (str) - Текст постов канала (очищенный)
    web_names_set - список ссылок на внешние ресурсы, на которые ссылается канал
    telegram_names_set - список телеграм каналов на которые ссылантся канал
    set(lang) (set) - язык постов
    av_post_len - среднее значение символов в одном посте
    '''


    import requests
    from fake_useragent import UserAgent
    import random
    import time
    from bs4 import BeautifulSoup
    import re
    import os
    import io
    import json



    url = 'https://t.me/s/{}'.format(user_name)
    useragent = {'User-Agent': UserAgent().random}
    text = ''

    try:
        r = requests.get(url, headers=useragent)
        time.sleep(random.randrange(1, 10))
        text = get_text(r, user_name, Dir_Path)
        number = re.findall(r"data-post=\"{}\/(\d+)".format(user_name), r.text)
        # print(number)
        if not len(number) == 0:
            last_post_num = int(number[-1])
            post_num = last_post_num
            if last_post_num < (iter_number+1)*20:
                iter_number = last_post_num//20 - 1
            for items in range(iter_number):
                r = requests.get('https://t.me/s/{}?before={}'.format(user_name, post_num), headers=useragent)
                get_text(r,user_name + str(post_num), Dir_Path)
                time.sleep(random.randrange(1, 7))
                temp_posts = re.findall(r"data-post=\"{}\/(\d+)".format(user_name), r.text)
                post_num = int(temp_posts[0])-1

    except Exception as ex:
        print(ex)
        return

def get_text(r, filename, Dir_Path):
    # Извлечение текста
    from bs4 import BeautifulSoup
    import re
    import os
    
    text = ''
    text_post = ''
    soup = BeautifulSoup(r.text, 'lxml')
    popular = soup.find_all('div', {'class': 'tgme_widget_message_text'})
    iterr = 0
    for item in popular:
        soup_temp = BeautifulSoup(item.text, 'lxml')
        text_post = re.sub(r'https?:\/\/.*[\r\n]*', ' ', soup_temp.text, flags=re.MULTILINE)
        text_post = re.sub('Африканская инициатива | Подписаться', ' ',text_post)
        text_post = re.sub(filename, ' ',text_post)
        text = text + '\n '+ text_post 
        predicted_class = "None"
        confidence = 0.
        try:
            if text_post != '':
                predicted_class, confidence = predict_contry (text_post)
                #print('predicted_class = {}, confidence_class = {:.0f}%'.format(predicted_class, confidence_class*100))
                #if predicted_class != item or confidence_class <0.2:
                #    print(content)
        except Exception as ex:
            print(ex)   
        
        #print('RES -{}, {}, {}, {}%'.format((predicted_class==Dir_Path) and (confidence>0.6), predicted_class, Dir_Path, confidence))
        if (len(text_post) > 150) and (len(text_post) < 2000) and (lang_detect(text_post) == 'ru') and (confidence>0.75):
            try:
                if not os.path.exists('./datasets/telegram_Dataset/{}'.format(country_name_rus_T[predicted_class])):
                    os.makedirs('./datasets/telegram_Dataset/{}'.format(country_name_rus_T[predicted_class]))
                with open('./datasets/telegram_Dataset/{}/@{}_telegram.txt'.format(country_name_rus_T[predicted_class], filename+'_'+str(iterr)), "w", encoding="utf-8") as some_file:
                    print(text_post, file=some_file)
                    some_file.close()
                    iterr = iterr+1
            except Exception as ex:
                print(ex)
    return text

def lang_detect (text = ''):
    # Определение языка канала
    from bs4 import BeautifulSoup
    from langdetect import detect
    import re

    if text !='':
        try:
            lang = detect(text[400:])
            print('Язык поста {}'.format(lang))
            return lang
        except Exception as ex:
            print(ex)
            return 'no text'
    else:
        return 'no text'

def pipline(content):
    import string
    import re
    from tokenizer_exceptions import normalizer_exc_rus
    print(content[:200])
    content = normalizer_exc_rus(content).lower()
    spec_chars = string.punctuation + '\xa0«»\t—…'
    content = re.sub('\n', ' ', content)
    content = re.sub('\r', ' ', content)
    content = re.sub('Африканская инициатива | Подписаться', ' ', content)
    content = remove_chars_from_text(content, spec_chars)
    content = remove_chars_from_text(content, string.digits)
    content = " ".join(content.split())
    print(content[:200])
    return content

def remove_chars_from_text(text, chars):
    # return "".join([ch for ch in text if ch not in chars])
    content = ''
    for ch in text:
        if ch not in chars:
            content = content + ''.join(ch)
        else:
            content = content + ''.join(' ')
    return content




def main():
    import os


    next_iter_names = {
        # 'china': ['maslovasia', 'china80s', 'raspp_info', 'prchand', 'awaken_dragon', 'asiatica_ru'],
        # 'africa': ['zangaro', 'africablack', 'natasakado_official', 'dnobangui', 'africafordummies', 'westernafrica', 'meskob', 'catcherinsudan', 'africanists'],
        # 'middleeast': ['syriaagency', 'turk_gambit_ca', 'shatergaddafi', 'turkkulubu', 'mideastr', 'tangermanar', 'marocrus', 'arabstatesofgulf', 'assadstash', 'meastru', 'strana_tuaregov']
        # 'latamerica': ['pqntc1', 'privetfidel', 'altimurla','favelasemrus', 'pincheponchito', 'laprimaveradelpatriarca', 'lat_america', 'tupireport', 'venezuelanewsnetwork', 'sputnikmundo']
        # 'news_ru': ['boris_rozhin', 'rt_russian', 'infantmilitario', 'svarschiki', 'divgen'],
        # 'news_ua': ['rezident_ua', 'taynaya_kantselyariya', 'the_military_analytics', 'spletnicca'],
        #'IranPakistanAfganistan': ['IranPakistanAfganistan', 'afgbezparandzhi'],
        #'KNDR': ['RusEmbDPRK'],
        #'Iran': ['irandezhurniy', 'paxIranica', 'IranPakistanAfganistan'],
        #'Afganistan': ['afgbezparandzhi', 'IranPakistanAfganistan'],
        #'Ethiopia': ['Ethiopia_tezeta', 'meskob'],
        #'Pakistan': ['IranPakistanAfganistan'],
        #'China': ['china80s', 'raspp_info', 'prchand'],
        #'India': ['india_tv2020', 'ninerasas', 'indiareads', 'speciallassi', 'indiaanalytics', 'Indosphere', 'India_sangrahalaya'],
        #'Turkey': ['turkeyabout', 'turkkulubu'],
        #'Libya': ['ShaterGaddafi'],
        #'Morocco': ['tangermanar', 'marocrus'],
        #'Syria': ['NovostiDamask'],
        #'CAR': ['dnobangui'],# Центрально-африканская Республика
        #'Sudan': ['CatcherInSudan'],
        #'Argentina': ['argentinarusa'],
        #'Iran': ['IranPakistanAfganistan'],
        #'Afganistan': [ 'IranPakistanAfganistan'],
        #'IranPakistanAfganistan': ['IranPakistanAfganistan'],
        #'africa': ['africaninitiative', 'africablack', 'natasakado_official', 'dnobangui', 'africafordummies', 'westernafrica', 'meskob', 'catcherinsudan', 'africanists'],
        #'africa': ['africaninitiative'],
        'China': ['xinhua_news_agency_ru'],
        #'latamerica': ['pqntc1', 'privetfidel', 'altimurla','favelasemrus', 'pincheponchito','laprimaveradelpatriarca', 'lat_america']
                

    }

    country_name_rus_T = {'Афганистан': 'Afganistan', 'КНР': 'China', 'Центрально-Африканская Республика': 'CAR',
                          'Эфиопия ': 'Ethiopia', 'Индия': 'India', 'Иран': 'Iran', 'КНДР': 'KNDR', 'Ливия': 'Libya',
                          'Морокко': 'Morocco', 'Судан': 'Sudan', 'Сирия': 'Syria', 'Турция': 'Turkey', 'Аргентина': 'Argentina',
                          'ДРК (Конго)': 'DRK_Congo', 'Буркина-Фасо': 'Burkina Faso', 'Нигер': 'Niger', 'Мали': 'Mali',
                          'Сенегал': 'Senegal', 'Египет': 'Egipet', 'Гвинея': 'Gvineya', 'Венесуэла': 'Venesuela', 'Алжир': 'Algeria',
                          'Зимбабве': 'Zimbabve', 'Тунис': 'Tunis', 'Боливия': 'Boliviya', 'Бразилия': 'Braziliya', 'ЮАР': 'Yuar',
                          'Бурунди': 'Burundi', 'Руанда': 'Ruanda', 'Монголия': 'Mongoliya', 'Южная Корея': 'Respublika-koreya',
                          'Чили': 'Chili', 'Бангладеш': 'Bangladesh', 'Бенин': 'Benin', 'Габон': 'Gabon',
                           'Кения': 'Keniya', 'Индонезия': 'Indoneziya', 'Иордания': 'Iordaniya', 'Ирак': 'Irak',
                          'Камбоджа': 'Kambodzha', 'Куба': 'Kuba', 'Таиланд': 'Tailand', 'Уругвай': 'Urugvay', 'Камерун': 'Kamerun',
                          'Мадагаскар': 'Madagaskar', 'Мозамбик': 'Mozambik', 'Япония': 'Yaponiya'
                         }
    CL =[ "Afganistan","China","CAR", "Ethiopia",
          "India", "Iran","KNDR", "Libya", "Morocco", "Sudan",
          "Syria", "Turkey", "Argentina", "DRK_Congo", "Burkina Faso",
          "Niger", "Mali", "Senegal", "Egipet", "Gvineya", "Venesuela",
          "Algeria", "Zimbabve", "Tunis", "Boliviya", "Braziliya",
          "Yuar", "Burundi", "Ruanda", "Mongoliya", "Respublika-koreya",
         "Chili", 'Bangladesh', 'Benin', 'Gabon',
         'Keniya', 'Indoneziya','Iordaniya', 'Irak', 'Kambodzha', 'Kuba',
         'Tailand','Urugvay', 'Kamerun', 'Madagaskar', 'Mozambik', 'Yaponiya'
        ]
    #for country in CL:
    #    if not os.path.exists('./datasets/telegram_Dataset/{}'.format(country)):
    #        os.makedirs('./datasets/telegram_Dataset/{}'.format(country))
       
    for item in next_iter_names.keys():
        for items in next_iter_names[item]:
            # print('Канал {}'.format(items))
            get_telegram_adv(items, item)


In [69]:
main()

Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
No features in text.
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
No features in text.
No features in text.
No features in text.
No features in text.
Язык поста ru
Язык поста ru
Язык поста ru

C:\Users\user\AppData\Local\Temp\ipykernel_8084\807388379.py:64: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup_temp = BeautifulSoup(item.text, 'lxml')


No features in text.
No features in text.
No features in text.
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
No features in text.
Язык поста ru
Язык поста ru
No features in text.
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
No features in text.
No features in text.
No features in text.
Язык поста ru
No features in text.
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ca
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста en
Язык поста ru
Язык поста ru
Язык поста ru
No features in text.
Язык поста ru
No features in text.
No features in text.
No features in text.
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
No features in text.
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык поста ru
Язык п

In [70]:
country_name_rus = {"Afganistan":'Афганистан',"China":'КНР',"CAR":'Центрально-Африканская Республика',
                    "Ethiopia":"Эфиопия ","India":'Индия', "Iran":'Иран',"KNDR":'КНДР', "Libya":'Ливия',
                    "Morocco":'Морокко', "Sudan":'Судан', "Syria":'Сирия', "Turkey":'Турция',
                    "Argentina":'Аргентина', "DRK_Congo":'ДРК (Конго)', "Burkina Faso":'Буркина-Фасо',
                    "Niger":'Нигер', "Mali":'Мали', "Senegal":'Сенегал', "Egipet":'Египет', "Gvineya":'Гвинея',
                    "Venesuela":'Венесуэла', "Algeria":'Алжир', "Zimbabve":'Зимбабве', "Tunis":'Тунис',
                    "Boliviya":'Боливия', "Braziliya":'Бразилия', "Yuar":'ЮАР', "Burundi":'Бурунди',
                    "Ruanda":'Руанда', "Mongoliya":'Монголия', "Respublika-koreya": 'Южная Корея',
                    "Chili": 'Чили',  'Bangladesh':'Бангладеш', 'Benin':'Бенин', 'Gabon':'Габон',
                    'Ekvatorialnaya-gviney':'Экваториальная Гвинея', 'Keniya':'Кения',
                    'Indoneziya':'Индонезия','Iordaniya':'Иордания', 'Irak':'Ирак',
                    'Kambodzha':'Камбоджа', 'Kuba':'Куба', 'Tailand':'Таиланд', 'Urugvay':'Уругвай',
                    'Kamerun':'Камерун', 'Madagaskar':'Мадагаскар', 'Mozambik':'Мозамбик',
                    'Yaponiya':'Япония'                
                   }
country_name_rus_T = {}
for item in country_name_rus.keys():
    country_name_rus_T[country_name_rus[item]] = item

print (country_name_rus_T)

{'Афганистан': 'Afganistan', 'КНР': 'China', 'Центрально-Африканская Республика': 'CAR', 'Эфиопия ': 'Ethiopia', 'Индия': 'India', 'Иран': 'Iran', 'КНДР': 'KNDR', 'Ливия': 'Libya', 'Морокко': 'Morocco', 'Судан': 'Sudan', 'Сирия': 'Syria', 'Турция': 'Turkey', 'Аргентина': 'Argentina', 'ДРК (Конго)': 'DRK_Congo', 'Буркина-Фасо': 'Burkina Faso', 'Нигер': 'Niger', 'Мали': 'Mali', 'Сенегал': 'Senegal', 'Египет': 'Egipet', 'Гвинея': 'Gvineya', 'Венесуэла': 'Venesuela', 'Алжир': 'Algeria', 'Зимбабве': 'Zimbabve', 'Тунис': 'Tunis', 'Боливия': 'Boliviya', 'Бразилия': 'Braziliya', 'ЮАР': 'Yuar', 'Бурунди': 'Burundi', 'Руанда': 'Ruanda', 'Монголия': 'Mongoliya', 'Южная Корея': 'Respublika-koreya', 'Чили': 'Chili', 'Бангладеш': 'Bangladesh', 'Бенин': 'Benin', 'Габон': 'Gabon', 'Экваториальная Гвинея': 'Ekvatorialnaya-gviney', 'Кения': 'Keniya', 'Индонезия': 'Indoneziya', 'Иордания': 'Iordaniya', 'Ирак': 'Irak', 'Камбоджа': 'Kambodzha', 'Куба': 'Kuba', 'Таиланд': 'Tailand', 'Уругвай': 'Urugvay', 'К

In [71]:
import sys
import os
import hashlib

def chunk_reader(fobj, chunk_size=1024):
    """Generator that reads a file in chunks of bytes"""
    while True:
        chunk = fobj.read(chunk_size)
        if not chunk:
            return
        yield chunk

def check_for_duplicates(paths, hash=hashlib.sha1):
    hashes = {}
    #print (os.listdir(paths))        
    for filename in [name for name in sorted(os.listdir(paths)) ]:
        full_path = os.path.join(paths, filename)
        hashobj = hash()
        for chunk in chunk_reader(open(full_path, 'rb')):
            hashobj.update(chunk)
        file_id = (hashobj.digest(), os.path.getsize(full_path))
        duplicate = hashes.get(file_id, None)
        if duplicate:
            print ("Duplicate found: %s and %s" % (full_path, duplicate))
            os.remove(full_path)
        else:
            hashes[file_id] = full_path

In [ ]:
for country in 
    path = os.path.join('./datasets/', "telegram_Dataset", country)
    #remove_duplicates(path)
    check_for_duplicates(path)